# Covid-19 Vaccine Allocations and Case Counts Analysis
----

Written in the Python 3.7.9 Environment

By Tarak Patel, Nicole Lund and Anne Niemiec.

Common Jupyter cells have been notated for ease of merging after individuals their analysis.

Individual working files will be merged using https://pypi.org/project/nbmerge/

In [4]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load Dependencies

# Data Organization & Collection
import pandas as pd
import requests
import json
from census import Census

# Data Visualization & Analysis
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import gmaps

# General Navigation
import time
import os
import sys

In [10]:
# Define users api key file folder location
api_key_path = r"C:\Users\nlund\Documents\GitHub\untracked_files"

In [11]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Add API key file to system path
sys.path.append(api_key_path)

# Import Census API Key
from untracked_api_keys import census_key
c = Census(census_key, year=2019)

# Import Google API key
from untracked_api_keys import g_key

In [14]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load clean data for Case and Death Counts by state and time
    # This data combines:
        # United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv'
        # state_code.csv

c_d_withstate_df_path = r"..\data_manipulation\clean_data\c_d_withstate_df.csv"
cases_deaths_df = pd.read_csv(c_d_withstate_df_path)
cases_deaths_df.head()

,Unnamed: 0,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,...,new_death,pnew_death,created_at,consent_cases,consent_deaths,State/District,Abbreviation,Postal Code,Month,Year
0,0,1/22/2020,AK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3/26/2020 16:22,0,0,Alaska,Alaska,AK,1,2020
1,1,1/23/2020,AK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3/26/2020 16:22,0,0,Alaska,Alaska,AK,1,2020
2,2,1/24/2020,AK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3/26/2020 16:22,0,0,Alaska,Alaska,AK,1,2020
3,3,1/25/2020,AK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3/26/2020 16:22,0,0,Alaska,Alaska,AK,1,2020
4,4,1/26/2020,AK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3/26/2020 16:22,0,0,Alaska,Alaska,AK,1,2020


In [17]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load clean data for Vaccination Allocation Data by state and time
    # This data combines:
        # COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Pfizer.csv
        # COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Moderna.csv
        # COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Janssen.csv

Combined_Vaccine_df_path = r"..\data_manipulation\clean_data\Combined_Vaccine_df.csv"
vaccine_allocation_df = pd.read_csv(Combined_Vaccine_df_path)
vaccine_allocation_df.head()

,Unnamed: 0,Jurisdiction,Week of Allocations,1st Dose Allocations_x,2nd Dose Allocations_x,1st Dose Allocations_y,2nd Dose Allocations_y,1st Dose Allocations
0,0,Connecticut,04/19/2021,54990,54990,40400.0,40400.0,NaN
1,1,Maine,04/19/2021,21060,21060,15400.0,15400.0,NaN
2,2,Massachusetts,04/19/2021,105300,105300,77700.0,77700.0,NaN
3,3,New Hampshire,04/19/2021,21060,21060,15500.0,15500.0,NaN
4,4,Rhode Island,04/19/2021,16380,16380,12100.0,12100.0,NaN


In [16]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load clean data for State Population from Census reports
    # This data collects census API request data by state:
        # Population.csv

Population_path = r"..\data_manipulation\clean_data\Population.csv"
census_df = pd.read_csv(Population_path)
census_df.head()

,Unnamed: 0,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count,Unemployment Rate
0,0,Alabama,4876250.0,39.0,50536.0,27928.0,795989.0,16.323794,132095.0,2.708946
1,1,Alaska,737068.0,34.3,77640.0,36787.0,76933.0,10.437707,26808.0,3.637114
2,2,Arizona,7050299.0,37.7,58945.0,30694.0,1043764.0,14.804535,195905.0,2.778676
3,3,Arkansas,2999370.0,38.1,47597.0,26577.0,496260.0,16.545475,70481.0,2.349860
4,4,California,39283497.0,36.5,75235.0,36955.0,5149742.0,13.109174,1199233.0,3.052765


# Group 2 Analysis